In [36]:
import folium
import pandas as pd

def create_marker(row, sub_category_icons, category_colors):
    """
    Create a Folium marker based on a row of data.

    Parameters:
    - row: Pandas DataFrame row
    - sub_category_icons: Dictionary mapping sub-categories to icons
    - category_colors: Dictionary mapping categories to colors

    Returns:
    - folium.Marker
    """
    category = row['Category']
    sub_category = row['Sub-Category']
    name = row['Name']
    lat = row['latitude']
    lon = row['longitude']
    color = category_colors.get(category, 'gray')  # Default color if category not found
    icon = sub_category_icons.get(sub_category, 'info-sign')  # Default icon if sub-category not found

    # Generate the HTML code for the pop-up dynamically (you may need to define html here)
    html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <link href="https://fonts.googleapis.com/css2?family=Poppins:wght@400;700&display=swap" rel="stylesheet">
        <style>
            body {{
                font-family: 'Poppins', sans-serif;
            }}
            .popup-container {{
                text-align: center;
            }}
        </style>
    </head>
    <body>
        <h1 align="center" style="font-family:Poppins; color:Blue"><strong><u>{row['Name']}</u><strong></h1>
        <div class="popup-container">
            <div class="panel-group" id="accordion" role="tablist" aria-multiselectable="true">
                <div class="panel panel-default" style="text-align: left; border-color: #ffffff !important;">
                    <div class="panel-heading" role="tab" id="headingOne" style="background: #ffffff!important;font-family:Poppins">
                        <h4 class="panel-title" style="color:cornflowerblue">
                            <a role="button" data-toggle="collapse" data-parent="#accordion" href="#collapseOne" aria-expanded="false" aria-controls="collapseOne">
                                <strong> <i class="fa fa-newspaper-o"></i> About <span class="glyphicon glyphicon-menu-down" aria-hidden="true" style="float: right;"></span></strong>
                            </a>
                        </h4>
                    </div>
                    <div id="collapseOne" class="panel-collapse collapse in" role="tabpanel" aria-labelledby="headingOne">
                        <div class="panel-body" align="justify" style="font-family:Poppins">
                            {row['Info']}
                        </div>
                    </div>
                </div>
                <div class="panel panel-default" style="text-align: left !important; border-color: #ffffff!important;">
                    <div class="panel-heading" role="tab" id="headingThree" style="background: #ffffff!important;font-family:Poppins">
                        <h4 class="panel-title" style="color:cornflowerblue">
                            <a class="collapsed" role="button" data-toggle="collapse" data-parent="#accordion" href="#collapseThree" aria-expanded="false" aria-controls="collapseThree">
                                <strong><i class="fa fa-map-marker"></i> Location <span class="glyphicon glyphicon-menu-down" aria-hidden="true" style="float: right;"></span> </strong>
                            </a>
                        </h4>
                    </div>
                    <div id="collapseThree" class="panel-collapse collapse" role="tabpanel" aria-labelledby="headingThree">
                        <div class="panel-body" align="justify" style="font-family:Poppins">
                            {row['Address']}
                        </div>
                    </div>
                </div>
                <div class="panel panel-default" style="text-align: left !important; border-color: #ffffff!important;">
                    <div class="panel-heading" role="tab" id="headingFour" style="background: #ffffff!important;font-family:Poppins">
                        <h4 class="panel-title" style="color:cornflowerblue">
                            <a class="collapsed" role="button" data-toggle="collapse" data-parent="#accordion" href="#collapseFour" aria-expanded="false" aria-controls="collapseFour">
                                <strong><i class="fa fa-clock-o"></i> Opening hours <span class="glyphicon glyphicon-menu-down" aria-hidden="true" style="float: right;"></span> </strong>
                            </a>
                        </h4>
                    </div>
                    <div id="collapseFour" class="panel-collapse collapse" role="tabpanel" aria-labelledby="headingFour">
                        <div class="panel-body" align="justify" style="font-family:Poppins">
                        <div>
                            <strong>Monday:</strong> {row['Monday']}
                        </div>
                        <div>
                            <strong>Tuesday:</strong> {row['Tuesday']}
                        </div>
                        <div>
                            <strong>Wednesday:</strong> {row['Wednesday']}
                        </div>
                        <div>
                            <strong>Thursday:</strong> {row['Thursday']}
                        </div>
                        <div>
                            <strong>Friday:</strong> {row['Friday']}
                        </div>
                        <div>
                            <strong>Saturday:</strong> {row['Saturday']}
                        </div>
                        <div>
                            <strong>Sunday:</strong> {row['Sunday']}
                        </div>
                        </div>
                    </div>
                </div>
            </div>
            <div class="popup-container" style="font-family:Poppins">
                <a href="{row['Website']}" target="_blank">Visit Website</a>  <!-- Add the hyperlink with the appropriate href attribute -->
            </div>
        </div>
    </body>
    </html>
    """

    # Create a marker with the corresponding color and icon
    marker = folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(color=color, icon=icon, prefix='fa'),
        tooltip=name,
        popup=folium.Popup(html, max_width=500)
    )

    return marker

def create_folium_map(locations):
    """
    Create a Folium map with layers and controls based on a dataframe.

    Parameters:
    - locations: Pandas DataFrame containing location data

    Returns:
    - folium.Map
    """
    # Create a Folium map
    map = folium.Map(location=[55.02540749519414, -1.5409688068645544], zoom_start=12)

    # Define a dictionary to map sub-categories to icons
    sub_category_icons = {
        'Food': 'cutlery',
        'Plastics': 'recycle',
        'Furniture': 'home',
        'Bikes': 'bicycle',
        'Homeware': 'shopping-basket',
        'Books': 'book',
        'Clothes': 'gift',
        'Toys': 'rocket',
        'Equipment': 'wheelchair',
        'Gardening' : 'leaf'
        # Add more sub-categories and icons as needed
    }

    # Define a dictionary to map categories to colors
    category_colors = {
        'Reduce': 'green',
        'Repair': 'red',
        'Reuse': 'darkblue'
        # Add more categories and colors as needed
    }

    # Create separate feature groups for each sub-category
    sub_category_groups = {}

    # Generate the HTML code for the legend
    legend_html = """
    <div style="position: fixed; bottom: 10px; right: 10px; z-index: 1000; background-color: white; padding: 10px; border: 2px solid grey;font-family:Poppins">
        <h4></h4>
        <ul style="list-style-type: none; padding: 0;">
            <li><i style="background: green; padding: 5px;"></i>- Reduce</li>
            <li><i style="background: red; padding: 5px;"></i>- Repair</li>
            <li><i style="background: darkblue; padding: 5px;"></i>- Reuse</li>
            <!-- Add more categories and colors as needed -->
        </ul>
    </div>
    """

    # Iterate over each row of your dataframe and add markers to the corresponding sub-category group
    for _, row in locations.iterrows():
        sub_category = row['Sub-Category']

        # Create a marker
        marker = create_marker(row, sub_category_icons, category_colors)

        # Check if the sub-category group already exists
        if sub_category not in sub_category_groups:
            # Create a new feature group for the sub-category
            group = folium.FeatureGroup(name=sub_category, show=False)  # Set show=False to hide initially
            sub_category_groups[sub_category] = group
            group.add_to(map)

        # Add the marker to the corresponding sub-category group
        group = sub_category_groups[sub_category]
        marker.add_to(group)

    # Add the sub-category groups to the map
    for _, group in sub_category_groups.items():
        group.add_to(map)

    # Add the legend to the map
    map.get_root().html.add_child(folium.Element(legend_html))

    # Add the layer control to the map to toggle the sub-category filter
    folium.LayerControl(collapsed=False, autoZIndex=False).add_to(map)

    return map

# Example usage
locations = pd.read_csv('Circular_Locations.csv')
folium_map = create_folium_map(locations)


In [37]:
# Display the map
folium_map

In [38]:
# Save the map to an HTML file
folium_map.save('Circular Economy Map.html')